# для определения тематики книги по названию

one-to-all

import json
import pandas as pd
import numpy as np

In [1]:
import nltk
import re
import pandas as pd
import numpy as np
from nltk.corpus import words, stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer, ISRIStemmer, LancasterStemmer, PorterStemmer, RegexpStemmer, RSLPStemmer, SnowballStemmer
from nltk.corpus import words as nltk_words

In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import *
from sklearn.utils import resample # будем делать downsumpling


In [3]:
from gensim.models import Word2Vec, KeyedVectors
#w2v_model = Word2Vec.load('enwv\en.model')
w2v_model = KeyedVectors.load_word2vec_format('enwv\GoogleNews-vectors-negative300.bin', binary= True)

C:\Users\blacat\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
import collections

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Embedding, Dropout, Conv1D, Dense, Flatten, LSTM, Bidirectional
from keras.layers.pooling import GlobalMaxPool1D, MaxPooling1D, MaxPool1D
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

C:\Users\blacat\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
#расширим список стоп-слов
## встречаются буквы по отдельности (окочания после апострофа и тп), возможно ошибки написания
mystopwords = set(stopwords.words('english') + ['a', 'b', 'c', 'd', 'e', 'f', 'j', 'h', 
           'g', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z']) 

eng_let = ['a', 'b', 'c', 'd', 'e', 'f', 'j', 'h', 'i', 
           'g', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

In [6]:
lemm = WordNetLemmatizer()
def tokenize(x):
    article = re.compile(u'[a-z]+')    
    return article.findall(x)

def stop_words(x):
    return [token for token in x if not token in mystopwords]

def lemma(x):
    return [lemm.lemmatize(token) for token in x ]

In [7]:
books = pd.read_csv('title.csv')

C:\Users\blacat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,9,11,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
allbooks = len(books)
allbooks

58751

In [9]:
subj_list = ['children', 'fantasy','medicine', 'music','mystery and detective stories', 
             'recipes', 'religion', 'romance', 'science','science fiction']

In [10]:
# число книг с определенной темой
b_num = {}
for i in subj_list:
    books[i].sum()
    b_num[i] = books[i].sum()
b_num

#subj_sum = books.religion.sum()
#subj_sum

{'children': 4235.0,
 'fantasy': 4384.0,
 'medicine': 3144.0,
 'music': 6077.0,
 'mystery and detective stories': 2316.0,
 'recipes': 1225.0,
 'religion': 11098.0,
 'romance': 10181.0,
 'science': 13222.0,
 'science fiction': 3770.0}

In [11]:
books[subj_list].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58751 entries, 0 to 58750
Data columns (total 10 columns):
children                         58751 non-null float64
fantasy                          58751 non-null float64
medicine                         58751 non-null float64
music                            58751 non-null float64
mystery and detective stories    58751 non-null float64
recipes                          58751 non-null float64
religion                         58751 non-null float64
romance                          58751 non-null float64
science                          58751 non-null float64
science fiction                  58751 non-null float64
dtypes: float64(10)
memory usage: 4.5 MB


In [12]:
books[subj_list].describe()

,children,fantasy,medicine,music,mystery and detective stories,recipes,religion,romance,science,science fiction
count,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000,58751.000000
mean,0.072084,0.074620,0.053514,0.103437,0.039421,0.020851,0.188899,0.173291,0.225051,0.064169
std,0.258629,0.262779,0.225058,0.304531,0.194595,0.142886,0.391432,0.378502,0.417620,0.245056
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
books.drop_duplicates(inplace = True)

In [14]:
books[subj_list].sum()

children                          4235.0
fantasy                           4384.0
medicine                          3144.0
music                             6077.0
mystery and detective stories     2316.0
recipes                           1225.0
religion                         11098.0
romance                          10181.0
science                          13222.0
science fiction                   3770.0
dtype: float64

In [15]:
X = books['title']
y = books[subj_list]

In [16]:
X_train, X_test, ytr, ytst = train_test_split(X, y, test_size=0.2, random_state=888, shuffle = True)

In [17]:
len(X_train)


47000

In [18]:
len(X_test)

11751

In [19]:
counter = collections.Counter()
maxlen = 0    
for key, value in X.items():
    title_words = stop_words(lemma(tokenize(value)))
    len_title = len(title_words)
    for w in title_words:
        counter[w] += 1
    if len_title>maxlen:
        maxlen = len_title

all_w = len(counter) # число всех слов в названиях книг
print('All words: ' + str(all_w) ,'. Max title length: '+ str(maxlen))
    
VOCAB_SIZE = all_w #используем все слова для создания эмбеддингов для лучшего результата. 
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 128
NUM_EPOCHS = 20
    
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid
vocab_sz = len(word2index) 
index2word = {v:k for k, v in word2index.items()}
        

print('Vocabular size: '+ str(vocab_sz))
    
embedd_wts = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
        #print(word, index)
    try:
        embedd_wts[index, :] = w2v_model[word]
    except KeyError:
        pass

    
Xtrain, Xtest = [], []
for i in X_train:
    title_words = stop_words(lemma(tokenize(i)))
    wids = [word2index[word] for word in title_words]
    Xtrain.append(wids)
    
for i in X_test:
    title_words = stop_words(lemma(tokenize(i)))
    wids = [word2index[word] for word in title_words]
    Xtest.append(wids)
        
Xtr = np.array(pad_sequences(Xtrain , maxlen = maxlen))
Xtst = np.array(pad_sequences(Xtest, maxlen = maxlen)) 

All words: 26968 . Max title length: 41
Vocabular size: 26968


# Реализация нейронной сети

LSTM

In [47]:
md = Sequential()
md.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen, weights = [embedd_wts], trainable=False))
md.add(LSTM(256, input_shape=(vocab_sz, EMBED_SIZE),  activation= 'relu' ))
md.add(Dropout(0.15))
#md.add(LSTM(128 , return_sequences=True , activation= 'relu' ))
#md.add(Dropout(0.15))
#md.add(LSTM(64, activation= 'relu' ))
#md.add(Dropout(0.15))
md.add(Dense(10, activation= 'softmax'))
#optimizer = RMSprop(1e-4)
md.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
checkp = ModelCheckpoint(filepath='title_weights\weights.best.title.hdf5', 
                               verbose=1, save_best_only=True)

In [56]:
process = md.fit(Xtr, ytr, batch_size = 512,epochs=10, validation_split=0.15, callbacks=[checkp])

Train on 39950 samples, validate on 7050 samples
Epoch 1/10
39950/39950 [==============================] - 161s 4ms/step - loss: 0.1215 - acc: 0.9549 - val_loss: 0.1614 - val_acc: 0.9426
Epoch 2/10
39950/39950 [==============================] - 160s 4ms/step - loss: 0.1230 - acc: 0.9544 - val_loss: 0.1620 - val_acc: 0.9425
Epoch 3/10
39950/39950 [==============================] - 165s 4ms/step - loss: 0.1522 - acc: 0.9536 - val_loss: 0.1630 - val_acc: 0.9430
Epoch 4/10
39950/39950 [==============================] - 168s 4ms/step - loss: 0.1174 - acc: 0.9564 - val_loss: 0.1636 - val_acc: 0.9422
Epoch 5/10
39950/39950 [==============================] - 160s 4ms/step - loss: 0.1174 - acc: 0.9565 - val_loss: 0.1625 - val_acc: 0.9436
Epoch 6/10
39950/39950 [==============================] - 161s 4ms/step - loss: 0.1122 - acc: 0.9586 - val_loss: 0.1642 - val_acc: 0.9430
Epoch 7/10
39950/39950 [==============================] - 159s 4ms/step - loss: 0.1105 - acc: 0.9593 - val_loss: 0.1666 - v

In [57]:
history = process.history
classes = process.model.predict_classes(Xtst, batch_size = 128, verbose = 1)
proba = process.model.predict_proba(Xtst, batch_size = 128)
print();

11751/11751 [==============================] - 18s 2ms/step



In [55]:
pr = np.around(proba)
pr[:20]
report = classification_report(ytst, pr)
print(report)

             precision    recall  f1-score   support

          0       0.81      0.46      0.59       850
          1       0.61      0.26      0.36       853
          2       0.76      0.57      0.65       619
          3       0.90      0.60      0.72      1206
          4       0.79      0.40      0.53       484
          5       0.95      0.78      0.85       273
          6       0.81      0.67      0.74      2243
          7       0.71      0.67      0.69      1985
          8       0.90      0.74      0.81      2625
          9       0.65      0.18      0.28       770

avg / total       0.80      0.59      0.67     11908



In [46]:
report = classification_report(ytst, pr)
print(report)

             precision    recall  f1-score   support

          0       0.86      0.39      0.54       850
          1       0.61      0.19      0.30       853
          2       0.79      0.48      0.60       619
          3       0.91      0.59      0.72      1206
          4       0.84      0.30      0.44       484
          5       0.93      0.68      0.79       273
          6       0.85      0.63      0.72      2243
          7       0.82      0.49      0.61      1985
          8       0.86      0.76      0.81      2625
          9       0.71      0.06      0.11       770

avg / total       0.83      0.53      0.62     11908



In [70]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (11, 8)

In [65]:
fpr = {}
tpr = {}
thr = {}
roc_auc = {}
#for i in range(0, 10):
#    fpr[i] , tpr[i] , thr[i] = roc_curve(ytst[i],proba[i])
#    roc_auc[i] = auc( fpr[i], tpr[i] )

In [ ]:
for subj in subj_list:
    plt.plot( fpr[subj], tpr[subj], label=subj +'. Area = {:.2f}'.format( roc_auc[subj]))       
    
plt.plot([0, 1], [0, 1])
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-AUC. Title Classificator')
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
for subj in subj_list:
    plt.plot(np.array(range(1, 36)), history_subj[subj]['loss'], \
             label= subj )
 
plt.title('Loss along Еpoch, distinct Subjects')
plt.legend(loc="upper right")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()
fig.savefig('Loss along Еpoch LSTM', dpi =300)
plt.show()

In [ ]:
for subj in subj_list:
    plt.plot(np.array(range(1, 36)), history_subj[subj]['acc'], \
             label=subj)
plt.title('Accuracy along Epoch, distinct Subjects')
plt.legend(loc="lower right")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid()
plt.show()